In [1]:
import preamble
import sounddevice as sd
import queue
import json
from vosk import Model, KaldiRecognizer
from robot import Robot
import pickle
from model import DualAutoencoderModel
from timer import Timer
from virtual_dynamics import SimpleVirtualDynamics
import numpy as np
from scipy.signal import savgol_filter
from plot_client import PlotClient
from arbitrary_sentence_warpper import ARBITRARY_SENTENCE_MAPPER

plot = PlotClient()

plot.create_plot('force_profile', title='Force vs Time', xlabel='Time (s)', ylabel='Force (N)')
plot.create_line("force_profile", "force_x", color="r", label="$F_x$")
plot.create_line("force_profile", "force_y", color="g", label="$F_y$")
plot.create_line("force_profile", "force_z", color="b", label="$F_z$")

plot_timer = Timer()

DISPLAY_TIME_WINDOW = 10

FORCE_SAMPLE_COUNT = 256
FORCE_CURVE_DURATION = 4.0

def derivative_smooth(curve):
    curve = savgol_filter(curve, window_length=50, polyorder=3, axis=0, mode='nearest')
    return np.gradient(curve, FORCE_CURVE_DURATION / FORCE_SAMPLE_COUNT, axis=0)

model = Model("../models/vosk-model-en-us-0.42-gigaspeech")
q = queue.Queue()

def callback(indata, frames, time, status):
    if status:
        print(status)
    q.put(bytes(indata))

samplerate = 16000

ROBOT_IP = '192.168.1.104'
INIT_POSE = [-0.54, -0.27, 0.0 + 0.02, -1.21, -1.21, 1.25]

F2L_MODEL_PATH = '../models/DAE_g.pkl'
with open(F2L_MODEL_PATH, 'rb') as file:
    F2L_model: DualAutoencoderModel = pickle.load(file)

force_profile_timer = Timer()

dynamics = SimpleVirtualDynamics(M=10, B=30, K=0)

print('lets go')

with Robot(
    ROBOT_IP,
    default_axes=Robot.TRANSLATION,
    init_pose=INIT_POSE,
) as r, sd.RawInputStream(
    samplerate=samplerate,
    blocksize=8000,
    dtype='int16',
    channels=1,
    callback=callback
):
    rec = KaldiRecognizer(model, samplerate)
    try:
        while True:
            data = q.get()
            arbitrary_phrase = None
            if rec.AcceptWaveform(data):
                command: str = json.loads(rec.Result())['text']
                print('command: ', command)
                command_words = command.split(' ')
                if len(command_words) >= 3 and ' '.join(command_words[:2]) == 'hey robot':
                    arbitrary_phrase = ' '.join(command_words[2:])
                    print('arbitrary_phrase: ',arbitrary_phrase)
                else:
                    plot_t = plot_timer.t()
                    plot.update_line("force_profile", "force_x", (plot_t, 0))
                    plot.update_line("force_profile", "force_y", (plot_t, 0))
                    plot.update_line("force_profile", "force_z", (plot_t, 0))
                    plot.config_plot("force_profile", xlim=(plot_t - DISPLAY_TIME_WINDOW, plot_t))

                    continue
            else:
                plot_t = plot_timer.t()
                plot.update_line("force_profile", "force_x", (plot_t, 0))
                plot.update_line("force_profile", "force_y", (plot_t, 0))
                plot.update_line("force_profile", "force_z", (plot_t, 0))
                plot.config_plot("force_profile", xlim=(plot_t - DISPLAY_TIME_WINDOW, plot_t))

                continue

            mvv_phrase = ARBITRARY_SENTENCE_MAPPER.map(arbitrary_phrase)
            print('mvv_phrase: ', mvv_phrase)
            if np.all(mvv_phrase == ""):
                continue
            phrase = np.array([mvv_phrase[0], mvv_phrase[1], mvv_phrase[2]], dtype='U16')
            phrase[1:] = np.sort(phrase[1:])
            force_profile = derivative_smooth(F2L_model.phrase_to_force(phrase[None, :])[0])

            force_profile_timer.reset()
            while force_profile_timer.t() < FORCE_CURVE_DURATION:
                period_start = r.control.initPeriod()
                dt = force_profile_timer.dt()

                index = int(force_profile_timer.t() * FORCE_SAMPLE_COUNT / FORCE_CURVE_DURATION)
                force = force_profile[index, :]
                dynamics.apply_force(force, dt)
                r.set_velocity(dynamics.get_velocity(), acceleration=2)

                r.control.waitPeriod(period_start)

                plot_t = plot_timer.t()
                plot.update_line("force_profile", "force_x", (plot_t, force[0]))
                plot.update_line("force_profile", "force_y", (plot_t, force[1]))
                plot.update_line("force_profile", "force_z", (plot_t, force[2]))
                plot.config_plot("force_profile", xlim=(plot_t - DISPLAY_TIME_WINDOW, plot_t))

            r.set_velocity(r.zeroed_wrench(), acceleration=10)

            while True:
                period_start = r.control.initPeriod()

                plot_t = plot_timer.t()
                plot.update_line("force_profile", "force_x", (plot_t, 0))
                plot.update_line("force_profile", "force_y", (plot_t, 0))
                plot.update_line("force_profile", "force_z", (plot_t, 0))
                plot.config_plot("force_profile", xlim=(plot_t - DISPLAY_TIME_WINDOW, plot_t))

                position = r.get_pose()
                position_delta = r.get_axes(r.INIT_POSE) - position
                position_delta_magnitude = np.linalg.norm(position_delta)
                position_delta_normalized = position_delta / position_delta_magnitude if position_delta_magnitude != 0 else 0
                return_velocity = 0.25
                r.set_velocity(return_velocity*position_delta_normalized, acceleration=2)
                r.control.waitPeriod(period_start)

                if position_delta_magnitude < 0.001:
                    r.set_velocity(r.zeroed_wrench(), acceleration=10)
                    break
    finally:
        pass

/Users/kv/.pyenv/versions/3.11.7/envs/shared-language-force-embedding-demo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=8
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:6:7:8:9:10
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from ../models/vosk-model-en-us-0.42-gigaspeech/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:279)

lets go
command:  hey robot harshly up
arbitrary_phrase:  harshly up
mvv_phrase:  ['harshly' 'up' '']
command:  
command:  
command:  hereby move down than sharply forward
command:  
command:  a bit of everything suffered the first two words hey robot moved down then sharply forward
command:  like fuck it it's just what we went what does that sentence jesus
command:  oh
command:  one thing you can do is like search for the first instance of a robot and then just cut everything before that because that might actually we might actually need that if there's a lot of people talking in the background and stuff


KeyboardInterrupt: 